In [1]:
raw_prompt = """
For each item that the buyer agreed to purchase, return the item name, item description, quantity purchased, unit price in dollars, unit discount in dollars, and final total price per item (quantity * (unit price - unit discount)) . Each sku of the purchased item should be treated as a separate item.

Output the result in dictionary format following the example below. Each line item should have its own list under lineItems. Include all non quantity and price related details of each item under its description.

{ "orderDetails":{ "orderDate": "", "orderNum": null, }, "buyer":{ "companyName": "", "pointOfContact": "", "contactEmail": "", "contactPhone": "", "shippingAddress": "", }, "lineItems":[{ "itemNumber": null, "description": "", "quantity": null, "unitPrice": null, "discount": null, “totalPrice”: null}]}
"""

In [2]:
from midas.agent import Midas

In [3]:
new_agent = Midas()

In [4]:
new_agent.set_objective(raw_prompt)

In [5]:
new_agent.add_criteria(
    [
        {"format": "the format should be outputted as a single string representing a python dictionary"},
        {"no_markdown": "the output should not be in markdown, it should not contain backticks"}
    ]
)

In [20]:
new_agent.save('/Users/kaihayden/Desktop/midas/first_subquery_save.json')

In [7]:
new_agent

Midas(
    Agent(
        name=,
        language_model=,
        embedding_model=
    )
    Prompt(
        raw=
For each item that the buyer agreed to purchase, return the item name, item description, quantity purchased, unit price in dollars, unit discount in dollars, and final total price per item (quantity * (unit price - unit discount)) . Each sku of the purchased item should be treated as a separate item.

Output the result in dictionary format following the example below. Each line item should have its own list under lineItems. Include all non quantity and price related details of each item under its description.

{ "orderDetails":{ "orderDate": "", "orderNum": null, }, "buyer":{ "companyName": "", "pointOfContact": "", "contactEmail": "", "contactPhone": "", "shippingAddress": "", }, "lineItems":[{ "itemNumber": null, "description": "", "quantity": null, "unitPrice": null, "discount": null, “totalPrice”: null}]}
,
        mod=
For each item that the buyer agreed to purchase, r

In [17]:
criteria_str = 'Criteria to follow:\n'
for name, struct in (new_agent.criteria.data['UserCriteria'] | new_agent.criteria.data['AgentCriteria']).items():
    criteria_str += f" - {name}: {struct.mod}\n"

In [19]:
print(criteria_str)

Criteria to follow:
 - format: the format should be outputted as a single string representing a python dictionary
 - no_markdown: the output should not be in markdown, it should not contain backticks
 - accuracy_of_item_details: The output accurately reflects each item's name, description, quantity purchased, unit price, and unit discount as specified in the user request.
 - completeness_of_order_details: The order details section is fully populated with the order date and order number.
 - completeness_of_buyer_information: The buyer section includes the company name, point of contact, contact email, contact phone, and shipping address.
 - correct_calculation_of_total_price_per_item: The final total price per item is correctly calculated as quantity * (unit price - unit discount).
 - correct_format_of_output: The output is in the correct dictionary format as specified, with each line item having its own list under lineItems.
 - inclusion_of_all_requested_fields: All requested fields (i

In [10]:
new_agent.criteria.data['UserCriteria']

{'format': Criteria(format), 'no_markdown': Criteria(no_markdown)}

In [9]:
new_agent.criteria.data['AgentCriteria']

{'accuracy_of_item_details': Criteria(accuracy_of_item_details),
 'completeness_of_order_details': Criteria(completeness_of_order_details),
 'completeness_of_buyer_information': Criteria(completeness_of_buyer_information),
 'correct_calculation_of_total_price_per_item': Criteria(correct_calculation_of_total_price_per_item),
 'correct_format_of_output': Criteria(correct_format_of_output),
 'inclusion_of_all_requested_fields': Criteria(inclusion_of_all_requested_fields),
 'correct_data_types': Criteria(correct_data_types),
 'no_redundant_or_unnecessary_information': Criteria(no_redundant_or_unnecessary_information)}

In [12]:
new_agent.criteria.data['AgentCriteria']['accuracy_of_item_details'].mod

"The output accurately reflects each item's name, description, quantity purchased, unit price, and unit discount as specified in the user request."

In [13]:
new_agent.criteria.data['AgentCriteria']['correct_format_of_output'].mod

'The output is in the correct dictionary format as specified, with each line item having its own list under lineItems.'

In [8]:
from openai import OpenAI

In [9]:
client = OpenAI()

In [25]:
completion = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
        {"role": "system", "content": CRITERIA_CONTEXT},
        {"role": "user", "content": raw_prompt}
    ],
    temperature=0
)

In [28]:
import json

In [35]:
json.loads(completion.choices[0].message.content)

{'accuracy_of_item_details': 'Each line item should accurately reflect the item name, description, quantity purchased, unit price, unit discount, and final total price per item.',
 'completeness_of_order_details': 'The order details must include the order date and order number.',
 'completeness_of_buyer_information': 'Buyer information should be complete, including company name, point of contact, contact email, contact phone, and shipping address.',
 'correct_calculation_of_total_price': 'The final total price per item should be correctly calculated using the formula quantity * (unit price - unit discount).',
 'correct_format_of_output': 'The output should be in the specified dictionary format, with each line item having its own list under lineItems and all non-quantity and price related details of each item under its description.',
 'presence_of_all_required_fields': 'Ensure all required fields are present: orderDetails, buyer, and lineItems with their respective details.',
 'correct_

In [6]:
new_agent.load('/Users/kaihayden/Desktop/midas/first_subquery_save.json')